In [1]:
import pandas as pd
from openpyxl import load_workbook
from IPython.display import Image
from sklearn.preprocessing import Imputer
from sklearn import tree
from subprocess import check_call
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score
#mengimport library yg digunakan untuk load data excel dan library lainnya yg digunakan untuk proses clustering

In [2]:
wb = pd.read_excel('datafix.xlsx', 'Sheet1')

df =wb
df.sample(10)
#load data excel yg akan digunakan untuk diclustering

SNo  BookingDate        ID          STTNo                 Shipper  \
1139  110810591  18-Apr-2018  JOG-1932   11-18-619525                   NISYA   
1610  110941405  07-May-2018  JOG-1932  10-17-1618409         MUH IKBAL HAKIM   
1219  110833283  21-Apr-2018  JOG-1932   11-18-640006  ARDYAN RISQI IRJAYANDY   
3512  111659656  28-Jul-2018  JOG-1932  11-18-1269128             EZE 19 SHOP   
2704  111268574  12-Jun-2018  JOG-1932   11-18-983371                    DEWI   
3115  111458032  10-Jul-2018  JOG-1932  11-18-1109993                    RAMA   
101   110490579  06-Mar-2018  JOG-1932   11-18-343624  MOH ERTON PATI GUSTIAN   
974   110763406  11-Apr-2018  JOG-1932   11-18-577362  PANTI RACER YOGYAKARTA   
3848  111833011  11-Aug-2018  JOG-1932  11-18-1404076            OUTLET GREEN   
113   110492995  06-Mar-2018  JOG-1932   11-18-345922             EZE 19 SHOP   

      OriginCity DestinationCity  ForwardArea  Product  GrossWt  VolumeWt  \
1139  JOGJAKARTA    PALANGKARAYA          NaN  REGPACK        1         1   
1610  JOGJAKARTA      BALIKPAPAN          NaN  ONEPACK        3         2   
1219  JOGJAKARTA        JAYAPURA          NaN  REGPACK        2         1   
3512  JOGJAKARTA        JAYAPURA          NaN  REGPACK        1         1   
2704  JOGJAKARTA         JAKARTA          NaN  REGPACK        1         1   
3115  JOGJAKARTA         MAKASAR          NaN  REGPACK        1         1   
101   JOGJAKARTA          LOMBOK          NaN  REGPACK        1         3   
974   JOGJAKARTA          SORONG          NaN  REGPACK        1         1   
3848  JOGJAKARTA         TARAKAN          NaN  REGPACK        2         1   
113   JOGJAKARTA        JAYAPURA          NaN  REGPACK        1         1   

         CommodityName            LastTrackingStatus  
1139  CLOTHING/GARMENT                 Return To POS  
1610        DRIED FOOD            RECEIPT BY BASRI .  
1219  CLOTHING/GARMENT         RECEIPT BY JEFRI K. .  
3512  CLOTHING/GARMENT  RECEIPT BY RESKY MARSELLIN .  
2704  CLOTHING/GARMENT    RECEIPT BY WIDI HARNIATI .  
3115       ACCESSORIES        RECEIPT BY A ROSNELI .  
101        ACCESSORIES           RECEIPT BY SOPYAN .  
974   CLOTHING/GARMENT           RECEIPT BY BOMBOM .  
3848  CLOTHING/GARMENT              RECEIPT BY EMA .  
113   CLOTHING/GARMENT             RECEIPT BY BRAM .

In [3]:


d = df[['SNo', 'Product', 'GrossWt', 'VolumeWt']]
d = d.dropna(axis=0, how='any')
d.columns = ['SNo', 'Product', 'Gross', 'Volume']
d
#menentukan baris dan kolom yg digunakan dalam proses clustering

SNo  Product  Gross  Volume
0     110459075  REGPACK      7       3
1     110459088  REGPACK      1       1
2     110459140  REGPACK      2       1
3     110459171  ONEPACK      1       1
4     110459198  REGPACK      1       1
5     110459247  REGPACK      1       1
6     110459865  REGPACK      1       1
7     110460042  REGPACK      1       1
8     110461292  REGPACK     22      17
9     110462311  REGPACK      4       2
10    110462342  REGPACK      6       4
11    110462345  REGPACK      5       2
12    110462371  REGPACK      2       1
13    110462399  REGPACK      3       1
14    110462478  REGPACK      1       1
15    110462486  REGPACK      1       1
16    110462529  REGPACK      1       1
17    110463490  REGPACK      1       1
18    110463636  REGPACK     15       1
19    110463694  REGPACK      5       2
20    110463743  REGPACK     15       2
21    110463822  REGPACK      1       1
22    110463917  REGPACK      4       2
23    110463938  REGPACK      1       2
24    110463983  REGPACK      2       2
25    110463987  ONEPACK      1       1
26    110464029  REGPACK      1       1
27    110465897  ONEPACK     31      25
28    110466442  REGPACK     14       6
29    110466532  REGPACK      1       1
...         ...      ...    ...     ...
4294  112050885  REGPACK      1       1
4295  112054444  REGPACK      8       1
4296  112054522  REGPACK      4       2
4297  112055450  REGPACK      1       1
4298  112055598  REGPACK      1       2
4299  112055661  REGPACK      1       1
4300  112056183  REGPACK      3       3
4301  112056223  REGPACK      6       4
4302  112058308  REGPACK      2       1
4303  112058352  REGPACK     10       8
4304  112058896  REGPACK      1       1
4305  112059080  REGPACK      1       1
4306  112059124  REGPACK      1       1
4307  112059250  REGPACK      1       1
4308  112059253  REGPACK      1       1
4309  112059399  REGPACK      2       1
4310  112059605  REGPACK      1       1
4311  112059616  REGPACK      6       5
4312  112059642  REGPACK      1       1
4313  112059645  REGPACK      3       2
4314  112059705  REGPACK      2       1
4315  112059780  REGPACK      1       1
4316  112060620  REGPACK      1       1
4317  112061510  REGPACK      7       8
4318  112061796  REGPACK      6       4
4319  112061999  REGPACK      5       4
4320  112062448  REGPACK      1       1
4321  112062913  REGPACK      1       1
4322  112064013  REGPACK      1       1
4323  112064220  ONEPACK      5       2

[4324 rows x 4 columns]

In [4]:
d.loc[d['Product'] == 'REGPACK', 'Product'] = 1
d.loc[d['Product'] == 'ONEPACK', 'Product'] = 2  
d.sample(10)

SNo  Product  Gross  Volume
1572  110928208        1      2       1
3888  111850431        1      1       1
4268  112043932        1      2       1
1445  110892416        1      4       3
2120  111083326        1      1       1
1429  110889538        1      2       2
2756  111301013        1      1       1
2545  111220104        1      1       1
1682  110965610        1     22      14
166   110511335        1      1       1

## Something something

In [5]:
from __future__ import division, print_function
import numpy as np
import matplotlib.pyplot as plt
import skfuzzy as fuzz


datasample = d.sample(400)

ModuleNotFoundError: No module named 'skfuzzy'

In [ ]:
proda = datasample.loc[datasample.Product == 1]
prodb = datasample.loc[datasample.Product == 2]

In [ ]:
colors = ['b', 'orange', 'g', 'r', 'c', 'm', 'y', 'k', 'Brown', 'ForestGreen']


# Generate test data
np.random.seed(23)  # Set seed for reproducibility
xpts = np.zeros(1)
ypts = np.zeros(1) 
labels = np.zeros(1)

for i, x in proda.iterrows():
    xapts = np.hstack((xpts, np.random.standard_normal(100) * x['Volume'] ))
    yapts = np.hstack((ypts, np.random.standard_normal(100) * x["Gross"]))
    
xpts = np.zeros(1)
ypts = np.zeros(1) 
for i, x in prodb.iterrows():
    xbpts = np.hstack((xpts, np.random.standard_normal(100) * x['Volume'] ))
    ybpts = np.hstack((ypts, np.random.standard_normal(100) * x["Gross"]))
    
# Visualize the test data 
alldata_a = np.vstack((xapts, yapts))
alldata_b = np.vstack((xbpts, ybpts))

ncluster = 2

cntr, u_orig, _, _, _, _, _ = fuzz.cluster.cmeans(
    alldata_a, 5, 2, error=0.005, maxiter=1000)

cntr, u_orig, _, _, _, _, _ = fuzz.cluster.cmeans(
    alldata_b, ncluster, 2, error=0.005, maxiter=1000)

u_orig

In [ ]:

# Show 3-cluster model
fig2, ax2 = plt.subplots(figsize =[8,8])
ax2.set_title('Trained model')
for j in range(ncluster):
    ax2.plot(alldata_a[0, u_orig.argmax(axis=0) == j],
             alldata_a[1, u_orig.argmax(axis=0) == j], 'o',
             label='ONEPACK ' + str(j))
    
for j in range(ncluster):
    ax2.plot(alldata_b[0, u_orig.argmax(axis=0) == j],
             alldata_b[1, u_orig.argmax(axis=0) == j], 'x',
             label='REGPACK ' + str(j))
    
ax2.legend()

In [ ]:
colors = ['b', 'orange', 'g', 'r', 'c', 'm', 'y', 'k', 'Brown', 'ForestGreen']


# Generate test data
np.random.seed(42)  # Set seed for reproducibility
xpts = np.zeros(1)
ypts = np.zeros(1)
labels = np.zeros(1)

for i, x in datasample.iterrows():
    xpts = np.hstack((xpts, np.random.standard_normal(100) * x['Volume'] ))
    ypts = np.hstack((ypts, np.random.standard_normal(100) * x["Gross"]))
    
# Visualize the test data 
alldata = np.vstack((xpts, ypts))

cntr, u_orig, _, _, _, _, _ = fuzz.cluster.cmeans(
    alldata, 3, 2, error=0.005, maxiter=1000)

# Show 3-cluster model
fig2, ax2 = plt.subplots()
ax2.set_title('Trained model')
symbols = ['o','x','s']
for j in range(3):
    ax2.plot(alldata[0, u_orig.argmax(axis=0) == j],
             alldata[1, u_orig.argmax(axis=0) == j], symbols[j],
             label='Cluster ' + str(j))
ax2.legend()